In [1]:

import sys
import os
sys.path.append(os.path.abspath(".."))
from core.viz import plot_line, create_subplot_grid, plot_bar, plot_statistical_strip
from core.s3 import S3AssetManager

import pandas as pd
import matplotlib.pyplot as plt

import numpy as np
from pathlib import Path
import seaborn as sns
import re, unicodedata


import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots



In [2]:
notebook_name = "comayma_adiflow"
s3 = S3AssetManager(notebook_name=notebook_name)
PALETTE =  [ "#1c8074","#1a494c", "#94af92", "#666666", "#f9ee77", "#f5ad68", "#c76931"]



In [3]:
df = s3.read_excel("raw/comayma/Análisis de datos (1).xlsx", sheet_name="Análisis")
df["Fórmula"] = df["Fórmula"].astype(str)
#df["horas"] = df["Quintales Producidos  QQ"]/df["qq/hora"]
df["horas"] = df["Cantidad plan (qq)"]/df["qq/hora"]
cond1 = df["Fórmula"] == "383"
cond2 = df["Adiflow"] == "Con"
#TODO: Se tiene que quitar el tiempo de paro
df.loc[(cond2 & cond1), "horas"] = 11.701608971233544 - 8.5
df["rendimiento"] = df["Cantidad plan (qq)"]/df["horas"]

In [4]:
df

,OP,Fórmula,Adiflow,Mes,Tiempo de Produccion Por OP(horas),Cantidad plan (qq),Quintales Producidos QQ,Merma,qq/hora,Tm/hora,...,Frecuencia de Alimentacion (Hz),Durabilidad (PDI) Control Proceso,Durabilidad (PDI) Calidad Producto Terminado,Humedad Entrada,Humedad Salida,Humedad P.T.,A.W.,Diametro de Pellet,horas,rendimiento
0,1071097,383,Sin,septiembre,02:47:32,720,708,12,259.73,11.81,...,44.30,86,88,12.85,14.25,12.41,0.69,4,2.772109,259.730000
1,1071347,383,Sin,septiembre,03:45:59,720,712,8,192.71,8.76,...,35.74,84,85,12.95,13.85,12.21,0.68,4,3.736184,192.710000
2,1071412,383,Sin,septiembre,03:24:28,720,710,10,212.35,9.65,...,32.75,89,84,12.05,14.30,11.56,0.68,4,3.390629,212.350000
3,1071506,383,Sin,septiembre,02:52:19,720,695,25,252.12,11.46,...,41.82,85,84,13.25,14.25,11.41,0.69,4,2.855783,252.120000
4,1071668,383,Sin,septiembre,03:14:00,720,711,9,223.94,10.18,...,45.02,88,88,13.25,14.25,11.69,0.68,4,3.215147,223.940000
5,1071882,383,Sin,septiembre,04:18:06,720,706,14,168.69,7.67,...,36.52,85,80,13.24,14.10,12.35,0.68,4,4.268184,168.690000
6,1072017,383,Sin,septiembre,03:10:27,720,710,10,228.58,10.39,...,39.07,80,83,13.00,14.00,12.47,0.68,4,3.149882,228.580000
7,1072018,383,Sin,septiembre,08:01:50,720,711,9,90.17,4.10,...,15.80,93,90,11.79,13.59,12.39,0.69,4,7.984917,90.170000
8,1072019,383,Sin,septiembre,03:05:38,720,707,13,234.12,10.64,...,38.52,85,90,13.05,13.95,12.06,0.68,4,3.075346,234.120000
9,1072410,383,Sin,septiembre,02:50:12,720,710,10,255.16,11.60,...,41.00,84,89,13.00,14.30,11.89,0.68,4,2.821759,255.160000


In [5]:
import plotly.express as px

fig = px.box(
    df[df["Humedad Entrada"]>0],
    x="Fórmula",
    y="Humedad Entrada",
    color="Adiflow",        # ← hue en plotly
    points="outliers"       # "all" para mostrar todos los puntos
)

# Opcional: orden y colores
fig.update_layout(
    legend_title_text="Adiflow",
    xaxis_title="Fórmula",
    yaxis_title="QQ por hora",
    margin=dict(l=20, r=20, t=40, b=20),
)

# Horizontal:
# fig = px.box(by_group, y="Fórmula", x="qq_hora", color="Adiflow", points="outliers")

fig.show()


In [6]:
def group_by(df, cols):

    df_ = df.groupby(["Adiflow"]+cols).agg(
        rows = ("Adiflow", "count"),
        cantidad_plan=("Cantidad plan (qq)", "sum"),
        cantidad_prod=("Quintales Producidos  QQ", "sum"),
        mermas=("Merma", "mean"),
        qq_hora=("qq/hora", "mean"),
        tm_hora=("Tm/hora", "mean"),
        ton_hora_teorico=('Teorico T/H', "median"),
        temp=("Temperatura  (°C) ", "median"),
        valula=("Valvula Reguladora BAR", "median"),
        amp_motor=("Amperaje Motor", "median"),
        kwh_t =("KWH/T", "mean"),
        pct_apertura_valvula=("% Apertura Valvula Reguladora","median"),
        freq_alimentador=( 'Frecuencia de Alimentacion  (Hz)  ', "median"),
        pdi_proceso=( 'Durabilidad  (PDI) Control Proceso','mean'),
        pdi_pt=( 'Durabilidad (PDI) Calidad Producto Terminado','mean'),
        hum_in=('Humedad Entrada', "median"),
        hum_out=('Humedad Salida', "median"),
        hum_pt =('Humedad P.T.', "median"),
        aw=('A.W.', "median" ),
        rendimiento=('rendimiento', "mean")).reset_index()

    return df_

In [7]:
group_by_tot = group_by(df, cols=[])
group_by_tot

,Adiflow,rows,cantidad_plan,cantidad_prod,mermas,qq_hora,tm_hora,ton_hora_teorico,temp,valula,...,kwh_t,pct_apertura_valvula,freq_alimentador,pdi_proceso,pdi_pt,hum_in,hum_out,hum_pt,aw,rendimiento
0,Con,4,2880,2836,11.000000,168.870000,7.677500,8.78,58.29,1.62,...,8.132500,41.365,38.605,85.250000,85.500000,13.16,14.055,12.30,0.685,209.709232
1,Sin,45,32400,31548,18.933333,203.128889,9.233778,9.64,56.17,1.38,...,11.783778,25.000,38.480,86.777778,87.022222,12.90,13.950,12.06,0.680,203.128889


In [8]:
by_group = group_by(df, cols=["Fórmula"])


In [9]:
# pip install pandas plotly kaleido
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from typing import List, Optional, Tuple

def grouped_bar_corporate_with_change(
    df: pd.DataFrame,
    formula_col: str = "Formula",             # eje X (categoría)
    group_col: str = "Adiflow",               # series: Con/Sin
    value_col: str = "mermas",                # valor
    title: str = "Mermas por fórmula y Adiflow",
    order_formula: Optional[List[str]] = None,
    order_group: Optional[List[str]] = None,  # por defecto ["Con","Sin"]
    # Colores corporativos:
    color_con: str = "#1C8074",
    color_sin: str = "#666666",
    up_color: str = "#1C8074",                # triángulo ↑
    down_color: str = "#c76931",              # triángulo ↓
    # Estilo:
    font_family: str = "Inter, 'Helvetica Neue', Arial, sans-serif",
    background: str = "#FFFFFF",
    text_fmt: str = ".1f",                    # formato etiqueta de barra
    show_bar_text: bool = True,
    # Cambio porcentual:
    show_change: bool = True,
    baseline: str = "Sin",                    # %Δ = (Con - Sin)/Sin
    pct_decimals: int = 1,
    marker_offset_ratio: float = 0.06,
    # >>> NUEVO: títulos y formato de ejes <<<
    x_axis_title: str = "Fórmula",
    y_axis_title: str = "Mermas",
    y_tickformat: Optional[str] = None,       # p.ej. ".1%" o ",.1f"
    # Guardado:
    write_html: Optional[str] = None,
    write_image: Optional[str] = None
) -> Tuple[go.Figure, pd.DataFrame, pd.DataFrame]:
    """
    Barras agrupadas Fórmula × Adiflow (Con/Sin) + cambio porcentual por fórmula.
    Ahora permite fijar títulos de ejes (x_axis_title, y_axis_title) y y_tickformat.
    Retorna (fig, df_plot, df_change).
    """
    # --- Validaciones ---
    for c in [formula_col, group_col, value_col]:
        if c not in df.columns:
            raise ValueError(f"Columna requerida no encontrada: {c}")

    # --- Agregación ---
    d = (
        df[[formula_col, group_col, value_col]]
        .dropna(subset=[formula_col, group_col, value_col])
        .groupby([formula_col, group_col], as_index=False)[value_col].sum()
    )

    # Orden
    if order_formula is not None:
        d[formula_col] = pd.Categorical(d[formula_col], categories=order_formula, ordered=True)
    if order_group is None:
        order_group = ["Con", "Sin"]
    d[group_col] = pd.Categorical(d[group_col], categories=order_group, ordered=True)
    d = d.sort_values([formula_col, group_col])

    # --- Barras ---
    fig = go.Figure()
    color_map = {"Con": color_con, "Sin": color_sin}
    for g in d[group_col].cat.categories:
        dd = d[d[group_col] == g]
        fig.add_trace(go.Bar(
            name=str(g),
            x=dd[formula_col],
            y=dd[value_col],
            marker_color=color_map.get(str(g), color_sin),
            marker_line_color="rgba(0,0,0,0.25)",
            marker_line_width=0.6,
            text=dd[value_col] if show_bar_text else None,
            texttemplate=f"%{{text:{text_fmt}}}",
            textposition="outside",
            hovertemplate=(
                f"{group_col}: <b>{g}</b><br>"
                f"{formula_col}: {{x}}<br>"
                f"{value_col}: {{y:{text_fmt}}}<extra></extra>"
            ),
        ))

    # --- Cambio porcentual por fórmula ---
    change_df = pd.DataFrame()
    if show_change:
        wide = d.pivot(index=formula_col, columns=group_col, values=value_col)

        def safe_pct(con, sin, base="Sin"):
            if base == "Sin":
                num, den = con, sin
                return np.nan if pd.isna(num) or pd.isna(den) or den == 0 else (con - sin)#/den
            else:
                num, den = sin, con
                return np.nan if pd.isna(num) or pd.isna(den) or den == 0 else (sin - con)#/den

        pct = []
        y_top = []
        for _, row in wide.iterrows():
            con, sin = row.get("Con", np.nan), row.get("Sin", np.nan)
            pct.append(safe_pct(con, sin, baseline))
            y_top.append(np.nanmax([con if pd.notna(con) else 0, sin if pd.notna(sin) else 0]))

        change_df = pd.DataFrame({formula_col: wide.index, "pct_change": pct, "y_top": y_top}).reset_index(drop=True)

        global_max = (d[value_col].max() or 0) if not d.empty else 0
        offset = max(1e-9, marker_offset_ratio * (global_max if global_max > 0 else 1.0))
        change_df["y_marker"] = change_df["y_top"] + offset

        def sym_col_text(v):
            if pd.isna(v):  return ("circle-open", "#666666", "")
            if v > 0:       return ("triangle-up", up_color, f"{v:.{pct_decimals}f} unidades")
            if v < 0:       return ("triangle-down", down_color, f"{v:.{pct_decimals}f} unidades")
            return ("circle", "#666666", "0.0%")

        tup = change_df["pct_change"].apply(sym_col_text)
        change_df["symbol"] = tup.apply(lambda t: t[0])
        change_df["color"]  = tup.apply(lambda t: t[1])
        change_df["text"]   = tup.apply(lambda t: t[2])

        fig.add_trace(go.Scatter(
            x=change_df[formula_col],
            y=change_df["y_marker"],
            mode="markers+text",
            marker=dict(symbol=change_df["symbol"], size=14, color=change_df["color"],
                        line=dict(color="#000000", width=0.6)),
            text=change_df["text"],
            textfont=dict(color="#000000"),
            textposition="top center",
            hovertemplate=(
                f"{formula_col}: {{x}}<br>"
                f"Δ ({'Con vs Sin' if baseline=='Sin' else 'Sin vs Con'}): "
                "{text}<extra></extra>"
            ),
            showlegend=False
        ))

    # --- Layout + marco ---
    fig.update_layout(
        title=f"<b>{title}</b>",
        barmode="group",
        bargap=0.25,
        bargroupgap=0.10,
        font=dict(family=font_family, size=12, color="#000000"),
        paper_bgcolor=background,
        plot_bgcolor=background,
        margin=dict(l=40, r=30, t=70, b=90),
        legend=dict(title=group_col, orientation="h", y=1.09, x=0),
        hoverlabel=dict(bgcolor="#FFFFFF", font=dict(color="#000000"), bordercolor="#666666"),
        shapes=[dict(
            type="rect", xref="paper", yref="paper",
            x0=0, y0=0, x1=1, y1=1,
            line=dict(color="#000000", width=1.2),
            fillcolor="rgba(0,0,0,0)"
        )]
    )

    # >>> títulos y formato de ejes <<<
    fig.update_xaxes(
        title_text=x_axis_title,
        showline=True, linecolor="#000000", linewidth=1,
        ticks="outside", tickcolor="#000000",
        tickangle=0, automargin=True
    )
    fig.update_yaxes(
        title_text=y_axis_title,
        showline=True, linecolor="#000000", linewidth=1,
        gridcolor="rgba(0,0,0,0.08)",
        zeroline=True, zerolinecolor="rgba(0,0,0,0.2)",
        automargin=True,
        tickformat=y_tickformat  # e.g., ".1%", ",.1f"
    )

    # Guardado
    if write_html:
        fig.write_html(write_html, include_plotlyjs="cdn")
    if write_image:
        fig.write_image(write_image, width=1400, height=720, scale=2)

    return fig, d, change_df


In [10]:
by_group

,Adiflow,Fórmula,rows,cantidad_plan,cantidad_prod,mermas,qq_hora,tm_hora,ton_hora_teorico,temp,...,kwh_t,pct_apertura_valvula,freq_alimentador,pdi_proceso,pdi_pt,hum_in,hum_out,hum_pt,aw,rendimiento
0,Con,312,1,720,712,8.000000,205.070000,9.320000,9.670,57.860,...,9.480000,17.00,37.690,78.000000,85.000000,13.050,14.050,12.380,0.69,205.070000
1,Con,313,1,720,707,13.000000,192.250000,8.740000,7.890,58.720,...,9.410000,40.00,35.820,84.000000,84.000000,13.100,14.530,12.220,0.68,192.250000
2,Con,383,1,720,707,13.000000,61.530000,2.800000,2.500,47.400,...,3.320000,1911.00,39.520,88.000000,85.000000,13.220,14.060,12.490,0.69,224.886926
3,Con,384,1,720,710,10.000000,216.630000,9.850000,10.460,64.330,...,10.320000,42.73,41.760,91.000000,88.000000,13.470,12.150,11.570,0.68,216.630000
4,Sin,312,11,7920,7752,15.272727,225.645455,10.256364,10.360,64.410,...,11.169091,25.00,40.760,87.090909,86.363636,12.950,13.900,12.090,0.68,225.645455
5,Sin,313,12,8640,8318,26.833333,211.667500,9.621667,9.880,57.660,...,12.998333,25.18,38.005,86.750000,88.000000,12.900,13.945,11.810,0.68,211.667500
6,Sin,383,12,8640,8499,11.750000,200.246667,9.103333,9.505,49.965,...,11.829167,20.00,37.520,85.750000,85.916667,12.975,14.050,12.115,0.68,200.246667
7,Sin,384,10,7200,6979,22.100000,171.573000,7.800000,8.660,48.205,...,10.948000,19.72,35.615,87.700000,87.900000,12.850,13.925,12.225,0.68,171.573000


In [11]:
group_by_tot

,Adiflow,rows,cantidad_plan,cantidad_prod,mermas,qq_hora,tm_hora,ton_hora_teorico,temp,valula,...,kwh_t,pct_apertura_valvula,freq_alimentador,pdi_proceso,pdi_pt,hum_in,hum_out,hum_pt,aw,rendimiento
0,Con,4,2880,2836,11.000000,168.870000,7.677500,8.78,58.29,1.62,...,8.132500,41.365,38.605,85.250000,85.500000,13.16,14.055,12.30,0.685,209.709232
1,Sin,45,32400,31548,18.933333,203.128889,9.233778,9.64,56.17,1.38,...,11.783778,25.000,38.480,86.777778,87.022222,12.90,13.950,12.06,0.680,203.128889


In [13]:
# pip install pandas plotly kaleido
import math
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from typing import List, Optional, Callable, Union, Dict


def bar_subplots_by_adiflow(
    df: pd.DataFrame,
    metrics: Optional[List[str]] = None,                    # ["mermas","qq_hora","tm_hora","temp","amp_motor","kwh_t"]
    group_col: str = "Adiflow",                             # categorías: Con / Sin
    agg: Union[str, Callable[[pd.Series], float]] = "mean", # "mean" | "sum" | "median" | función
    title: str = "Indicadores Globales con uso de Adiflow y su variación Δ",

    # Estética corporativa
    color_con: str = "#1C8074",
    color_sin: str = "#666666",
    font_family: str = "Inter, 'Helvetica Neue', Arial, sans-serif",
    background: str = "#FFFFFF",

    # Layout de subplots
    cols: int = 3,
    sharey: bool = False,                                   # ejes Y independientes

    # Etiquetas y formatos
    y_tickformat: Optional[str] = ",.2f",                   # p.ej. ".1%" si son proporciones
    bar_text_fmt: str = ",.2f",
    bar_text_color: str = "#000000",
    inside_values: bool = True,                             # valores centrados dentro de la barra

    # Cálculo de cambio por subplot (diferencia en unidades)
    baseline: str = "Sin",                                  # Δ = (Con - Sin) si baseline es "Sin" (o viceversa)
    pct_decimals: int = 1,
    triangle_up_color: str = "#1C8074",
    triangle_down_color: str = "#c76931",
    triangle_neutral_color: str = "#666666",
    triangle_offset_ratio: float = 0.06,                    # % del máximo del subplot para ubicar triángulo

    # >>> Nombres de ejes <<<
    x_axis_title: str = "Condición Adiflow",
    y_axis_titles: Optional[Union[str, Dict[str, str]]] = None,  # str para todos, o dict por métrica
    y_titles_mode: str = "all",                             # "all" | "first_col"

    # Mostrar/ocultar títulos por subplot
    show_subplot_titles: bool = False,

    # Guardado opcional
    write_html: Optional[str] = None,
    write_image: Optional[str] = None
) -> go.Figure:
    """
    Subplots de barras (uno por métrica) con X=Adiflow (Con/Sin), valores centrados en las barras,
    Δ (diferencia en unidades) entre grupos y ejes personalizables. Por defecto NO muestra títulos
    por subplot (show_subplot_titles=False).
    """

    # -------- Validaciones y defaults --------
    if group_col not in df.columns:
        raise ValueError(f"Columna '{group_col}' no encontrada en df.")

    if metrics is None:
        candidates = ["mermas", "qq_hora", "tm_hora", "temp", "amp_motor", "kwh_t", "pdi_pt"]
        metrics = [m for m in candidates if m in df.columns]
        if not metrics:
            raise ValueError("No se encontraron métricas. Pasa `metrics=[...]` explícitamente.")

    missing_metrics = [m for m in metrics if m not in df.columns]
    if missing_metrics:
        raise ValueError(f"Métricas no encontradas en df: {missing_metrics}")

    df = df.copy()
    for m in metrics:
        df[m] = pd.to_numeric(df[m], errors="coerce")

    # Orden de grupos preferido
    uniques = df[group_col].astype(str).unique().tolist()
    order_groups = [g for g in ["Con", "Sin"] if g in uniques] or sorted(uniques)

    aggfunc = agg

    # Layout de subplots
    n = len(metrics)
    rows = math.ceil(n / cols)
    fig = make_subplots(
        rows=rows, cols=cols,
        subplot_titles=(metrics if show_subplot_titles else None),
        horizontal_spacing=0.10,
        vertical_spacing=0.15,
        shared_yaxes=False if not sharey else "all"
    )

    # Leyenda: una sola (primer subplot)
    def show_legend_for(r, c):
        return (r == 1 and c == 1)

    color_map = {"Con": color_con, "Sin": color_sin}

    # Helper para título de Y por métrica
    def y_title_for(metric: str) -> Optional[str]:
        if y_axis_titles is None:
            return metric
        if isinstance(y_axis_titles, str):
            return y_axis_titles
        return y_axis_titles.get(metric, metric)

    # -------- Helper para cada métrica --------
    def add_subplot_for_metric(m: str, r: int, c: int):
        # Agregado por grupo
        d = (
            df[[group_col, m]]
            .dropna(subset=[m])
            .groupby(group_col, as_index=False)[m]
            .agg(aggfunc)
        )

        if d.empty:
            return

        d[group_col] = pd.Categorical(d[group_col].astype(str), categories=order_groups, ordered=True)
        d = d.sort_values(group_col)

        # Barras Con/Sin
        for grp in d[group_col].cat.categories:
            dd = d[d[group_col] == grp]
            if dd.empty:
                continue
            fig.add_trace(
                go.Bar(
                    name=str(grp),
                    legendgroup=str(grp),
                    showlegend=show_legend_for(r, c),
                    x=dd[group_col],
                    y=dd[m],
                    marker_color=color_map.get(str(grp), color_sin),
                    marker_line_color="rgba(0,0,0,0.25)",
                    marker_line_width=0.6,
                    # valores centrados dentro de la barra
                    texttemplate=f"%{{y:{bar_text_fmt}}}",
                    textposition="inside" if inside_values else "outside",
                    insidetextanchor="middle",
                    textfont=dict(color=bar_text_color),
                    hovertemplate=f"{group_col}: <b>{grp}</b><br>{m}: %{{y:{bar_text_fmt}}}<extra></extra>"
                ),
                row=r, col=c
            )

        # ---- Δ (diferencia en unidades) entre grupos ----
        s = d.set_index(group_col)[m]
        con_val = float(s.get("Con")) if "Con" in s.index else np.nan
        sin_val = float(s.get("Sin")) if "Sin" in s.index else np.nan

        def delta_units(con, sin, base="Sin"):
            if base == "Sin":
                return con - sin
            else:
                return sin - con

        ymax = float(np.nanmax(d[m])) if not d.empty else 0.0
        offset = max(1e-9, triangle_offset_ratio * (ymax if ymax > 0 else 1.0))
        y_marker = ymax + offset

        d_units = np.nan
        if not (pd.isna(con_val) or pd.isna(sin_val)):
            d_units = delta_units(con_val, sin_val, base=baseline)

        if pd.isna(d_units):
            tri_char, tri_color, txt = "●", triangle_neutral_color, ""
        else:
            tri_char = "▲" if d_units > 0 else ("▼" if d_units < 0 else "●")
            tri_color = triangle_up_color if d_units > 0 else (triangle_down_color if d_units < 0 else triangle_neutral_color)
            txt = f"{d_units:.{pct_decimals}f} unidades"

        axis_index = (r - 1) * cols + c
        xref = f"x{axis_index} domain" if axis_index > 1 else "x domain"
        yref = f"y{axis_index}" if axis_index > 1 else "y"

        fig.add_annotation(
            x=0.5, xref=xref, y=y_marker, yref=yref,
            text=f"{tri_char} {txt}", showarrow=False,
            font=dict(color=tri_color, size=12),
            align="center"
        )

        # Ejes por subplot
        fig.update_xaxes(
            title_text=x_axis_title,
            showline=True, linecolor="#000000", linewidth=1,
            ticks="outside", tickcolor="#000000",
            automargin=True,
            row=r, col=c
        )

        put_y_title = (y_titles_mode == "all") or (y_titles_mode == "first_col" and c == 1)
        fig.update_yaxes(
            title_text=y_title_for(m) if put_y_title else None,
            showline=True, linecolor="#000000", linewidth=1,
            gridcolor="rgba(0,0,0,0.08)",
            zeroline=True, zerolinecolor="rgba(0,0,0,0.2)",
            automargin=True,
            tickformat=y_tickformat,
            row=r, col=c
        )

    # -------- Render de todos los subplots --------
    r = c = 1
    for m in metrics:
        add_subplot_for_metric(m, r, c)
        c += 1
        if c > cols:
            c = 1
            r += 1

    # -------- Layout global --------
    fig.update_layout(
        title=f"<b>{title}</b>",
        barmode="group",
        bargap=0.25,
        bargroupgap=0.12,
        font=dict(family=font_family, size=12, color="#000000"),
        paper_bgcolor=background,
        plot_bgcolor=background,
        margin=dict(l=60, r=40, t=60 if not show_subplot_titles else 80, b=60),
        legend=dict(title=group_col, orientation="h", y=1.05, x=0),
        hoverlabel=dict(bgcolor="#FFFFFF", font=dict(color="#000000"), bordercolor="#666666"),
        shapes=[dict(  # marco
            type="rect", xref="paper", yref="paper",
            x0=0, y0=0, x1=1, y1=1,
            line=dict(color="#000000", width=1.2),
            fillcolor="rgba(0,0,0,0)"
        )]
    )

    # Asegura que los ejes Y NO estén enlazados si sharey=False
    if not sharey:
        fig.update_yaxes(matches=None)

    return fig


In [14]:
metrics = ["mermas", "rendimiento", "pdi_pt"]

fig = bar_subplots_by_adiflow(
    group_by_tot,
    metrics=metrics,
    agg="mean",
    title="Indicadores  Globales en Producción, Calidad y su variación Δ",
    cols=3,
    inside_values=True,
    bar_text_fmt=",.2f",
    bar_text_color="#000000",
    x_axis_title="Condición Adiflow",
    y_axis_titles={                      # etiqueta por métrica
        "mermas": "Mermas (qm)",
        "temp": "Temperatura (°C)",
        "rendimiento": "Rendimiento (qm/hora)",
        "pdi_pt": "PDI PT (%)"
    },
    y_titles_mode="all",                 # ← muestra el Y de todos los subplots
    #write_html=f"{ROOT_IMAGEN}/subplots_adiflow_mermas_temp.html"
)
fig.show()
s3.save_plotly_html(fig, "subplots_adiflow_mermas_temp.html")


In [15]:
metrics = ["amp_motor", "kwh_t", "freq_alimentador"]

fig = bar_subplots_by_adiflow(
    group_by_tot,
    metrics=metrics,
    agg="mean",
    title="Indicadores Globales del gasto energético y su variación Δ",
    cols=3,
    inside_values=True,
    bar_text_fmt=",.2f",
    bar_text_color="#000000",
    x_axis_title="Condición Adiflow",
    y_axis_titles={                      # etiqueta por métrica
        "amp_motor": "Amp",
        "kwh_t": "Consumo(Kwh/T)",
        "freq_alimentador": "Freq(Hz)"
    },
    y_titles_mode="all",                 # ← muestra el Y de todos los subplots
   # write_html=f"{ROOT_IMAGEN}/subplots_adiflow_energia.html"
)
fig.show()
s3.save_plotly_html(fig, "subplots_adiflow_energia.html")


In [16]:
metrics = [ 'hum_in', 'hum_out', 'hum_pt', "temp"]

fig = bar_subplots_by_adiflow(
    group_by_tot,
    metrics=metrics,
    agg="median",
    title="Indicadores  Globales en la Humedad  y Temperatura en Distintos Puntos y su variación Δ",
    cols=2,
    inside_values=True,
    bar_text_fmt=",.2f",
    bar_text_color="#000000",
    x_axis_title="Condición Adiflow",
    y_axis_titles={                      # etiqueta por métrica
        "hum_in": "Humedad Entrada (%)",
        "hum_out": "Humedad Salida (%)",
        "hum_pt": "Humedad PT (%)",
        "temp": "Temperatura (°C)"
    },
    y_titles_mode="all",                 # ← muestra el Y de todos los subplots
    #write_html=f"{ROOT_IMAGEN}/subplots_adiflow_hum.html"
)
fig.show()
s3.save_plotly_html(fig, f"subplots_adiflow_hum.html")


In [17]:
group_df = group_by_tot[["Adiflow", "rows", "cantidad_prod", "mermas", "rendimiento", "temp", "amp_motor", "kwh_t", "freq_alimentador", "pdi_pt",  'hum_in', 'hum_out', 'hum_pt', 'aw']].round(2)
s3.save_dataframe(group_df, "summary.csv")


In [18]:
orden_formula = [
    "312",
    "313",
    "383",
    "384"
]

fig, _, _ = grouped_bar_corporate_with_change(
    by_group,                                 # tu DataFrame
    formula_col="Fórmula",
    group_col="Adiflow",
    value_col="mermas",
    title="Mermas por fórmula y Adiflow",
    order_formula=orden_formula,
    order_group=["Con", "Sin"],         # asegura el orden
    color_con="#1C8074",                # verde corporativo
    color_sin="#666666",                # gris corporativo
     y_axis_title="Mermas (qm)",
    # Guardado opcional:
    #write_html=f"{ROOT_IMAGEN}/barras_mermas.html",
)

fig.show()
s3.save_plotly_html(fig, "barras_mermas.html")

In [19]:

fig, _, _ = grouped_bar_corporate_with_change(
    by_group,                                 # tu DataFrame
    formula_col="Fórmula",
    group_col="Adiflow",
    value_col="temp",
    title="Temperatura (°C)  por fórmula y Adiflow",
    order_formula=orden_formula,
    order_group=["Con", "Sin"],         # asegura el orden
    color_con="#1C8074",                # verde corporativo
    color_sin="#666666",                # gris corporativo
    # Guardado opcional:
    #write_html=f"{ROOT_IMAGEN}/barras_temp.html",
    x_axis_title="Fórmula ",
    y_axis_title="Temperatura (°C)",
)

fig.show()
s3.save_plotly_html(fig, "barras_temp.html")

In [20]:

fig, df_plot, df_change = grouped_bar_corporate_with_change(
    by_group,                                 # tu DataFrame
    formula_col="Fórmula",
    group_col="Adiflow",
    value_col="amp_motor",
    title="Amperios en el motor por fórmula y Adiflow",
    order_formula=orden_formula,
    order_group=["Con", "Sin"],         # asegura el orden
    color_con="#1C8074",                # verde corporativo
    color_sin="#666666",                # gris corporativo
    # Guardado opcional:
    #write_html=f"{ROOT_IMAGEN}/barras_amp_motor.html",
     y_axis_title="Amperios",
)

fig.show()
s3.save_plotly_html(fig, f"barras_amp_motor.html")

In [21]:
(8.13-11.78)

-3.6499999999999986

In [22]:

fig, df_plot, df_change = grouped_bar_corporate_with_change(
    by_group,                                 # tu DataFrame
    formula_col="Fórmula",
    group_col="Adiflow",
    value_col="kwh_t",
    title="Consumo energía en el motor por fórmula y Adiflow",
    order_formula=orden_formula,
    order_group=["Con", "Sin"],         # asegura el orden
    color_con="#1C8074",                # verde corporativo
    color_sin="#666666",                # gris corporativo
    # Guardado opcional:
    #write_html=f"{ROOT_IMAGEN}/barras_amp_motor.html",
    y_axis_title="Consumo (Kwh/Ton)",

)

fig.show()
s3.save_plotly_html(fig, f"barras_amp_motor.html")

In [23]:

fig, df_plot, df_change = grouped_bar_corporate_with_change(
    by_group,                                 # tu DataFrame
    formula_col="Fórmula",
    group_col="Adiflow",
    value_col="freq_alimentador",
    title="Frecuencia Alimentador  por fórmula y Adiflow",
    order_formula=orden_formula,
    order_group=["Con", "Sin"],         # asegura el orden
    color_con="#1C8074",                # verde corporativo
    color_sin="#666666",                # gris corporativo
    # Guardado opcional:
    #write_html=f"{ROOT_IMAGEN}/barras_freq.html",
    y_axis_title="Frecuencia Alimentador (Hz)",
)

fig.show()
s3.save_plotly_html(fig, f"barras_freq.html")

In [24]:

fig, df_plot, df_change = grouped_bar_corporate_with_change(
    by_group,                                 # tu DataFrame
    formula_col="Fórmula",
    group_col="Adiflow",
    value_col="pdi_pt",
    title="PDI PT por fórmula y Adiflow",
    order_formula=orden_formula,
    order_group=["Con", "Sin"],         # asegura el orden
    color_con="#1C8074",                # verde corporativo
    color_sin="#666666",                # gris corporativo
    # Guardado opcional:
    #write_html=f"{ROOT_IMAGEN}/pdi_pt.html",
    y_axis_title="PDI PT(%)",
)

fig.show()
s3.save_plotly_html(fig, f"pdi_pt.html")

In [25]:
by_group

,Adiflow,Fórmula,rows,cantidad_plan,cantidad_prod,mermas,qq_hora,tm_hora,ton_hora_teorico,temp,...,kwh_t,pct_apertura_valvula,freq_alimentador,pdi_proceso,pdi_pt,hum_in,hum_out,hum_pt,aw,rendimiento
0,Con,312,1,720,712,8.000000,205.070000,9.320000,9.670,57.860,...,9.480000,17.00,37.690,78.000000,85.000000,13.050,14.050,12.380,0.69,205.070000
1,Con,313,1,720,707,13.000000,192.250000,8.740000,7.890,58.720,...,9.410000,40.00,35.820,84.000000,84.000000,13.100,14.530,12.220,0.68,192.250000
2,Con,383,1,720,707,13.000000,61.530000,2.800000,2.500,47.400,...,3.320000,1911.00,39.520,88.000000,85.000000,13.220,14.060,12.490,0.69,224.886926
3,Con,384,1,720,710,10.000000,216.630000,9.850000,10.460,64.330,...,10.320000,42.73,41.760,91.000000,88.000000,13.470,12.150,11.570,0.68,216.630000
4,Sin,312,11,7920,7752,15.272727,225.645455,10.256364,10.360,64.410,...,11.169091,25.00,40.760,87.090909,86.363636,12.950,13.900,12.090,0.68,225.645455
5,Sin,313,12,8640,8318,26.833333,211.667500,9.621667,9.880,57.660,...,12.998333,25.18,38.005,86.750000,88.000000,12.900,13.945,11.810,0.68,211.667500
6,Sin,383,12,8640,8499,11.750000,200.246667,9.103333,9.505,49.965,...,11.829167,20.00,37.520,85.750000,85.916667,12.975,14.050,12.115,0.68,200.246667
7,Sin,384,10,7200,6979,22.100000,171.573000,7.800000,8.660,48.205,...,10.948000,19.72,35.615,87.700000,87.900000,12.850,13.925,12.225,0.68,171.573000


In [26]:

fig, df_plot, df_change = grouped_bar_corporate_with_change(
    by_group,                                 # tu DataFrame
    formula_col="Fórmula",
    group_col="Adiflow",
    value_col="rendimiento",
    title="Rendimiento por fórmula y Adiflow",
    order_formula=orden_formula,
    order_group=["Con", "Sin"],         # asegura el orden
    color_con="#1C8074",                # verde corporativo
    color_sin="#666666",                # gris corporativo
    # Guardado opcional:
    #write_html=f"{ROOT_IMAGEN}/c.html",
    y_axis_title="Rendimiento (qm/hora)",
)

fig.show()
s3.save_plotly_html(fig, f"rendimiento.html")

In [27]:
df

,OP,Fórmula,Adiflow,Mes,Tiempo de Produccion Por OP(horas),Cantidad plan (qq),Quintales Producidos QQ,Merma,qq/hora,Tm/hora,...,Frecuencia de Alimentacion (Hz),Durabilidad (PDI) Control Proceso,Durabilidad (PDI) Calidad Producto Terminado,Humedad Entrada,Humedad Salida,Humedad P.T.,A.W.,Diametro de Pellet,horas,rendimiento
0,1071097,383,Sin,septiembre,02:47:32,720,708,12,259.73,11.81,...,44.30,86,88,12.85,14.25,12.41,0.69,4,2.772109,259.730000
1,1071347,383,Sin,septiembre,03:45:59,720,712,8,192.71,8.76,...,35.74,84,85,12.95,13.85,12.21,0.68,4,3.736184,192.710000
2,1071412,383,Sin,septiembre,03:24:28,720,710,10,212.35,9.65,...,32.75,89,84,12.05,14.30,11.56,0.68,4,3.390629,212.350000
3,1071506,383,Sin,septiembre,02:52:19,720,695,25,252.12,11.46,...,41.82,85,84,13.25,14.25,11.41,0.69,4,2.855783,252.120000
4,1071668,383,Sin,septiembre,03:14:00,720,711,9,223.94,10.18,...,45.02,88,88,13.25,14.25,11.69,0.68,4,3.215147,223.940000
5,1071882,383,Sin,septiembre,04:18:06,720,706,14,168.69,7.67,...,36.52,85,80,13.24,14.10,12.35,0.68,4,4.268184,168.690000
6,1072017,383,Sin,septiembre,03:10:27,720,710,10,228.58,10.39,...,39.07,80,83,13.00,14.00,12.47,0.68,4,3.149882,228.580000
7,1072018,383,Sin,septiembre,08:01:50,720,711,9,90.17,4.10,...,15.80,93,90,11.79,13.59,12.39,0.69,4,7.984917,90.170000
8,1072019,383,Sin,septiembre,03:05:38,720,707,13,234.12,10.64,...,38.52,85,90,13.05,13.95,12.06,0.68,4,3.075346,234.120000
9,1072410,383,Sin,septiembre,02:50:12,720,710,10,255.16,11.60,...,41.00,84,89,13.00,14.30,11.89,0.68,4,2.821759,255.160000


In [28]:
desc = (
    df
      .groupby(["Adiflow", "Fórmula"], dropna=False)["qq/hora"]
      .describe(percentiles=[.05, .25, .5, .75, .95])
      .rename(columns={
          "std": "sd", "50%": "median", "5%": "q05",
          "25%": "q25", "75%": "q75", "95%": "q95"
      })
      .round(3)
)

# Si la quieres “tidy” (índices como columnas):
desc = desc.reset_index()
desc

,Adiflow,Fórmula,count,mean,sd,min,q05,q25,median,q75,q95,max
0,Con,312,1.0,205.070,NaN,205.07,205.070,205.070,205.070,205.070,205.070,205.07
1,Con,313,1.0,192.250,NaN,192.25,192.250,192.250,192.250,192.250,192.250,192.25
2,Con,383,1.0,61.530,NaN,61.53,61.530,61.530,61.530,61.530,61.530,61.53
3,Con,384,1.0,216.630,NaN,216.63,216.630,216.630,216.630,216.630,216.630,216.63
4,Sin,312,11.0,225.645,43.551,167.53,168.900,198.000,221.220,245.720,291.705,314.32
5,Sin,313,12.0,211.668,60.931,90.45,112.060,171.618,240.425,255.118,268.653,277.75
6,Sin,383,12.0,200.247,54.386,90.17,107.792,167.312,218.145,238.620,257.216,259.73
7,Sin,384,10.0,171.573,50.923,89.37,98.212,131.290,183.955,208.358,226.865,239.96
